In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install google-adk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12

In [3]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [4]:
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


Helper Functions

In [5]:
def show_python_code_and_result(response):
    for i in range(len(response)):
        # Check if the response contains a valid function call result from the code executor
        if (
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    print("Generated Python Response >> ", response_code["result"])


print("✅ Helper functions defined.")

✅ Helper functions defined.


Configure retry options

In [6]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

**Custom Tools**

In [7]:
# ICP Metals Tool
def analyze_icp_metals(sample_id: str, ni: float, v: float, fe: float) -> dict:
    """
    Analyzes ICP metals for a given lab sample and flags anomalies.

    Args:
        sample_id: Unique identifier for the sample.
        ni: Nickel concentration (ppm).
        v: Vanadium concentration (ppm).
        fe: Iron concentration (ppm).

    Returns:
        Dictionary with status and interpreted results.
        Success example: {"status": "success", "sample_id": "...", "results": {...}}
        Error example: {"status": "error", "error_message": "..."}
    """
    try:
        results = {}
        # Simple threshold checks for demonstration
        results['Ni'] = "Normal" if ni < 100 else "High"
        results['V'] = "Normal" if v < 50 else "High"
        results['Fe'] = "Normal" if fe < 200 else "High"

        return {"status": "success", "sample_id": sample_id, "results": results}

    except Exception as e:
        return {"status": "error", "error_message": str(e)}

print("✅ ICP Metals tool created")
print(f"Test: {analyze_icp_metals('SAMPLE123', 80, 30, 150)}")


✅ ICP Metals tool created
Test: {'status': 'success', 'sample_id': 'SAMPLE123', 'results': {'Ni': 'Normal', 'V': 'Normal', 'Fe': 'Normal'}}


In [8]:
# Physical Properties Tool
def analyze_physical_properties(sample_id: str, density: float, viscosity: float, flash_point: float) -> dict:
    """
    Analyzes physical properties of a lab sample.

    Args:
        sample_id: Unique identifier for the sample.
        density: Sample density (g/cm³).
        viscosity: Sample viscosity (cP).
        flash_point: Flash point (°C).

    Returns:
        Dictionary with status and interpreted results.
        Success example: {"status": "success", "sample_id": "...", "results": {...}}
        Error example: {"status": "error", "error_message": "..."}
    """
    try:
        results = {}
        results['Density'] = "Normal" if 0.8 <= density <= 1.0 else "Abnormal"
        results['Viscosity'] = "Normal" if 10 <= viscosity <= 50 else "Abnormal"
        results['Flash Point'] = "Normal" if flash_point >= 40 else "Low"

        return {"status": "success", "sample_id": sample_id, "results": results}

    except Exception as e:
        return {"status": "error", "error_message": str(e)}

print("✅ Physical Properties tool created")
print(f"Test: {analyze_physical_properties('SAMPLE123', 0.85, 25, 60)}")


✅ Physical Properties tool created
Test: {'status': 'success', 'sample_id': 'SAMPLE123', 'results': {'Density': 'Normal', 'Viscosity': 'Normal', 'Flash Point': 'Normal'}}


In [9]:
# Sulfur Analysis Tool
def analyze_sulfur(sample_id: str, sulfur_content: float) -> dict:
    """
    Analyzes sulfur content of a lab sample.

    Args:
        sample_id: Unique identifier for the sample.
        sulfur_content: Sulfur concentration (%) in the sample.

    Returns:
        Dictionary with status and interpreted results.
        Success example: {"status": "success", "sample_id": "...", "results": {...}}
        Error example: {"status": "error", "error_message": "..."}
    """
    try:
        results = {}
        results['Sulfur'] = "Normal" if sulfur_content <= 1.0 else "High"

        return {"status": "success", "sample_id": sample_id, "results": results}

    except Exception as e:
        return {"status": "error", "error_message": str(e)}

print("✅ Sulfur Analysis tool created")
print(f"Test: {analyze_sulfur('SAMPLE123', 0.8)}")


✅ Sulfur Analysis tool created
Test: {'status': 'success', 'sample_id': 'SAMPLE123', 'results': {'Sulfur': 'Normal'}}


In [10]:
# Lab Sample Agent

# Lab Sample Agent with custom function tools
lab_agent = LlmAgent(
    name="lab_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are a lab sample analysis assistant.

    For a given lab sample:
    1. Use `analyze_icp_metals()` to check Ni, V, Fe levels.
    2. Use `analyze_physical_properties()` to review density, viscosity, flash point.
    3. Use `analyze_sulfur()` to assess sulfur content.
    4. Check the "status" field in each tool's response for errors.
    5. Provide a consolidated summary of all results, highlighting any anomalies.
    """,
    tools=[analyze_icp_metals, analyze_physical_properties, analyze_sulfur],
)

print("✅ Lab Sample Agent created with custom function tools")
print("🔧 Available tools:")
print("  • analyze_icp_metals - Checks Ni, V, Fe concentrations")
print("  • analyze_physical_properties - Reviews density, viscosity, flash point")
print("  • analyze_sulfur - Assesses sulfur content")

# Test the lab agent
lab_runner = InMemoryRunner(agent=lab_agent)
_ = await lab_runner.run_debug(
    "Analyze lab sample SAMPLE123 with ICP metals: Ni=80, V=30, Fe=150; "
    "Physical properties: density=0.85, viscosity=25, flash_point=60; "
    "Sulfur content=0.8"
)


✅ Lab Sample Agent created with custom function tools
🔧 Available tools:
  • analyze_icp_metals - Checks Ni, V, Fe concentrations
  • analyze_physical_properties - Reviews density, viscosity, flash point
  • analyze_sulfur - Assesses sulfur content

 ### Created new session: debug_session_id

User > Analyze lab sample SAMPLE123 with ICP metals: Ni=80, V=30, Fe=150; Physical properties: density=0.85, viscosity=25, flash_point=60; Sulfur content=0.8


lab_agent > All analyses for sample SAMPLE123 are complete and show normal results for all tested parameters.


**Create a Calculation Agent**

In [11]:
# Calculation Agent - for lab calculations
calculation_agent = LlmAgent(
    name="CalculationAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are a specialized calculator for lab QC metrics. You MUST ONLY output Python code.
    
    RULES:
    1. Output only a single Python code block.
    2. Do NOT write any text before or after the code.
    3. Python code must perform the calculation requested.
    4. Print the final result using `print(result)` at the end.
    5. DO NOT perform calculations yourself; your job is only to generate executable code.
    """,
    code_executor=BuiltInCodeExecutor(),  # Executes Python code
)


**Update Lab Sample Agent to Use CalculationAgent**
Create a specialized agent that:
* Calls existing tools (analyze_icp_metals, analyze_physical_properties, analyze_sulfur).
* Delegates composite calculations (like QC scoring or threshold checks) to the calculation_agent.


In [12]:
# Enhanced Lab Sample Agent with Calculation Delegation
enhanced_lab_agent = LlmAgent(
    name="enhanced_lab_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are a lab sample analysis assistant. You must strictly follow these steps and use the available tools.

    For any lab sample analysis request:

    1. Analyze ICP Metals: Use `analyze_icp_metals()` to check Ni, V, Fe levels.
    2. Analyze Physical Properties: Use `analyze_physical_properties()` to review density, viscosity, and flash point.
    3. Analyze Sulfur: Use `analyze_sulfur()` to assess sulfur content.
    4. Error Check: After each tool call, check the "status" field in the response. If the status is "error", stop immediately and explain the issue.
    5. Complex Calculations (CRITICAL): You are strictly prohibited from performing any arithmetic or threshold computations yourself. Use the `calculation_agent` tool to compute derived metrics such as averages, totals, ratios, or anomaly flags.
    6. Provide Detailed Summary: After calculations, provide a complete summary including:
        * ICP Metals results
        * Physical properties
        * Sulfur content
        * Any derived metrics or flagged anomalies
    """,
    tools=[
        analyze_icp_metals,
        analyze_physical_properties,
        analyze_sulfur,
        AgentTool(agent=calculation_agent),  # Delegate complex calculations
    ],
)

print("✅ Enhanced Lab Sample Agent created")
print("🎯 New capability: Delegates complex lab calculations to specialist agent")
print("🔧 Tool types used:")
print("  • Function Tools (ICP metals, physical properties, sulfur)")
print("  • Agent Tool (calculation specialist)")

✅ Enhanced Lab Sample Agent created
🎯 New capability: Delegates complex lab calculations to specialist agent
🔧 Tool types used:
  • Function Tools (ICP metals, physical properties, sulfur)
  • Agent Tool (calculation specialist)


In [13]:
# Define a runner for the enhanced lab agent
enhanced_lab_runner = InMemoryRunner(agent=enhanced_lab_agent)

# Test the enhanced lab agent
response = await enhanced_lab_runner.run_debug(
    "Analyze lab sample SAMPLE123 with ICP metals: Ni=80, V=30, Fe=150; "
    "Physical properties: density=0.85, viscosity=25, flash_point=60; "
    "Sulfur content=0.8. Show me the full detailed summary including any calculations."
)



 ### Created new session: debug_session_id

User > Analyze lab sample SAMPLE123 with ICP metals: Ni=80, V=30, Fe=150; Physical properties: density=0.85, viscosity=25, flash_point=60; Sulfur content=0.8. Show me the full detailed summary including any calculations.


enhanced_lab_agent > Sample analysis for SAMPLE123:

ICP Metals:
- Ni: Normal
- V: Normal
- Fe: Normal

Physical Properties:
- Density: Normal
- Viscosity: Normal
- Flash Point: Normal

Sulfur Content:
- Sulfur Content: Normal

Derived Metrics:
- Ni/V Ratio: 2.80

Overall Status: ALL QC PASSED
```json
{
  "ICP Metals": {
    "Ni": "Normal",
    "V": "Normal",
    "Fe": "Normal"
  },
  "Physical Properties": {
    "Density": "Normal",
    "Viscosity": "Normal",
    "Flash Point": "Normal"
  },
  "Sulfur Content": {
    "Sulfur": "Normal"
  },
  "Derived Metrics": {
    "Ni/V Ratio": 2.80
  },
  "Overall Status": "ALL QC PASSED"
}
```


In [14]:
show_python_code_and_result(response)